<a href="https://colab.research.google.com/github/prithvikavoori/PySparkMllib/blob/main/LinearRegressionPysparkModel.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [36]:
# innstall java
!apt-get install openjdk-8-jdk-headless -qq > /dev/null

In this model we will examine some ecommerce customers data for companyny website and mobile app to predict a customer's total amount expenditure

--> we have to convert the data in to the format accepted by sparks MLlib

--> the data used in this model will be available with this model

Setting up Pyspark

In [37]:
# install spark (change the version number if needed)
!wget -q https://archive.apache.org/dist/spark/spark-3.0.0/spark-3.0.0-bin-hadoop3.2.tgz

In [38]:
# unzip the spark file to the current folder
!tar xf spark-3.0.0-bin-hadoop3.2.tgz

In [39]:
# set your spark folder to your system path environment. 
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.0.0-bin-hadoop3.2"

In [41]:
# install findspark using pip
!pip install -q findspark

In [42]:
import findspark
findspark.init()

In [43]:
from pyspark.sql import SparkSession

In [44]:
spark= SparkSession.builder.appName("lr_example").getOrCreate()

In [46]:
# read the data 
data=spark.read.csv("/content/Ecommerce_Customers.csv",inferSchema=True,header=True)

In [48]:
# to view the schema of the data 
data.printSchema()

root
 |-- Email: string (nullable = true)
 |-- Address: string (nullable = true)
 |-- Avatar: string (nullable = true)
 |-- Avg Session Length: double (nullable = true)
 |-- Time on App: double (nullable = true)
 |-- Time on Website: double (nullable = true)
 |-- Length of Membership: double (nullable = true)
 |-- Yearly Amount Spent: double (nullable = true)



In [49]:
data.head(1)

[Row(Email='mstephenson@fernandez.com', Address='835 Frank TunnelWrightmouth, MI 82180-9605', Avatar='Violet', Avg Session Length=34.49726772511229, Time on App=12.65565114916675, Time on Website=39.57766801952616, Length of Membership=4.0826206329529615, Yearly Amount Spent=587.9510539684005)]

In [50]:
for item in data.head(1)[0]:
  print(item)

mstephenson@fernandez.com
835 Frank TunnelWrightmouth, MI 82180-9605
Violet
34.49726772511229
12.65565114916675
39.57766801952616
4.0826206329529615
587.9510539684005


In [51]:
from pyspark.ml.linalg import Vectors
from pyspark.ml.feature import VectorAssembler

In [52]:
data.columns

['Email',
 'Address',
 'Avatar',
 'Avg Session Length',
 'Time on App',
 'Time on Website',
 'Length of Membership',
 'Yearly Amount Spent']

in the above data columns we are only interested in Avg Session Length',
 'Time on App',
 'Time on Website',
 'Length of Membership' to predict Yearly Amount Spent

In [53]:
assembler=VectorAssembler(inputCols=['Avg Session Length','Time on App','Time on Website','Length of Membership'],outputCol='features' )

In [54]:
#transforming the data 

output=assembler.transform(data)

In [55]:
output.printSchema()

root
 |-- Email: string (nullable = true)
 |-- Address: string (nullable = true)
 |-- Avatar: string (nullable = true)
 |-- Avg Session Length: double (nullable = true)
 |-- Time on App: double (nullable = true)
 |-- Time on Website: double (nullable = true)
 |-- Length of Membership: double (nullable = true)
 |-- Yearly Amount Spent: double (nullable = true)
 |-- features: vector (nullable = true)



In [56]:
output.head(1)

[Row(Email='mstephenson@fernandez.com', Address='835 Frank TunnelWrightmouth, MI 82180-9605', Avatar='Violet', Avg Session Length=34.49726772511229, Time on App=12.65565114916675, Time on Website=39.57766801952616, Length of Membership=4.0826206329529615, Yearly Amount Spent=587.9510539684005, features=DenseVector([34.4973, 12.6557, 39.5777, 4.0826]))]

In [57]:
final_data=output.select('features','Yearly Amount Spent')

In [59]:
final_data.show()

+--------------------+-------------------+
|            features|Yearly Amount Spent|
+--------------------+-------------------+
|[34.4972677251122...|  587.9510539684005|
|[31.9262720263601...|  392.2049334443264|
|[33.0009147556426...| 487.54750486747207|
|[34.3055566297555...|  581.8523440352177|
|[33.3306725236463...|  599.4060920457634|
|[33.8710378793419...|   637.102447915074|
|[32.0215955013870...|  521.5721747578274|
|[32.7391429383803...|  549.9041461052942|
|[33.9877728956856...|  570.2004089636196|
|[31.9365486184489...|  427.1993848953282|
|[33.9925727749537...|  492.6060127179966|
|[33.8793608248049...|  522.3374046069357|
|[29.5324289670579...|  408.6403510726275|
|[33.1903340437226...|  573.4158673313865|
|[32.3879758531538...|  470.4527333009554|
|[30.7377203726281...|  461.7807421962299|
|[32.1253868972878...| 457.84769594494855|
|[32.3388993230671...| 407.70454754954415|
|[32.1878120459321...|  452.3156754800354|
|[32.6178560628234...|   605.061038804892|
+----------

In [60]:
# spliting our data in train data and test data 
train_data,test_data=final_data.randomSplit([0.7,0.3])

In [61]:
train_data.describe().show()

+-------+-------------------+
|summary|Yearly Amount Spent|
+-------+-------------------+
|  count|                336|
|   mean| 500.59290269775556|
| stddev|  79.23038166364948|
|    min| 256.67058229005585|
|    max|  765.5184619388373|
+-------+-------------------+



In [62]:
test_data.describe().show()

+-------+-------------------+
|summary|Yearly Amount Spent|
+-------+-------------------+
|  count|                164|
|   mean| 496.69392574908284|
| stddev|  79.66587377355407|
|    min|  275.9184206503857|
|    max|  725.5848140556806|
+-------+-------------------+



In [64]:
# creating a linear regression model 
lr=LinearRegression(labelCol="Yearly Amount Spent")
lr_model=lr.fit(train_data)

In [65]:
#Evaluating a model 
test_results=lr_model.evaluate(test_data)
test_results.residuals.show()

+-------------------+
|          residuals|
+-------------------+
| -4.690168424876845|
|  6.405784420811187|
|-22.060001236933545|
| -4.087183937086593|
|  2.887681062571687|
| -4.359687132659246|
| 2.8220210520564137|
| -7.237471516969549|
| -3.258126668385387|
| -5.441577173006067|
|  3.973489505344162|
| -18.28401399152949|
|-14.834313198173504|
| 17.022651824096158|
| -7.052394068837771|
| -6.563652413876241|
|  2.161837165486588|
|   7.86385087753132|
| -9.182719178670311|
| 1.4094137497372685|
+-------------------+
only showing top 20 rows



In [66]:
test_results.rootMeanSquaredError

10.367356677462476

In [67]:
test_results.r2

0.982960859798049

In [68]:
final_data.describe().show()

+-------+-------------------+
|summary|Yearly Amount Spent|
+-------+-------------------+
|  count|                500|
|   mean|  499.3140382585909|
| stddev|   79.3147815497068|
|    min| 256.67058229005585|
|    max|  765.5184619388373|
+-------+-------------------+



In [70]:
#deploying the model for only features 

unlabeled_data=test_data.select("features")
unlabeled_data.show()
predictions=lr_model.transform(unlabeled_data)
predictions.show()

+--------------------+
|            features|
+--------------------+
|[30.8364326747734...|
|[30.9716756438877...|
|[31.1239743499119...|
|[31.2681042107507...|
|[31.3091926408918...|
|[31.4252268808548...|
|[31.4459724827577...|
|[31.4474464941278...|
|[31.5171218025062...|
|[31.5257524169682...|
|[31.5316044825729...|
|[31.5702008293202...|
|[31.5741380228732...|
|[31.6098395733896...|
|[31.7207699002873...|
|[31.7242025238451...|
|[31.7366356860502...|
|[31.8209982016720...|
|[31.8279790554652...|
|[31.8293464559211...|
+--------------------+
only showing top 20 rows

+--------------------+------------------+
|            features|        prediction|
+--------------------+------------------+
|[30.8364326747734...|472.19206885186645|
|[30.9716756438877...|488.23282533608153|
|[31.1239743499119...| 509.0070550766993|
|[31.2681042107507...| 427.5577171109105|
|[31.3091926408918...|429.83303677736194|
|[31.4252268808548...| 535.1264057874212|
|[31.4459724827577...|482.05494388307216|
|[